# Dynamic Programming and Structural Econometrics #4b

### Portfolio Choice Example:  Numerical Integration

**Readings:** 
- 📖 Judd, K. L. (1998). Numerical methods in economics. MIT press. Sections 7.6

by Bertel Schjerning

University of Copenhagen

### Numerical methods in Economics

Optimization problems and integrals frequently arise in economics and econometrics

**Numerical Integration**:
- Expected utility/profits: Integral over stochastic states of the world
- Discounted utility and profits over a long horizon in continuous time 
- Moments (mean, variance, skewness, etc), Likelihood functions, Bayesian posterior
- Solution methods for dynamic economic models

**Optimization**: 
- Maximization of utility, profits, social welfare, tax-revenue, equality...
- Minimization of expenses, cost, commute time, carbon emissions...
- Estimation: Least squares, maximum likelihood...

*Most integrals and optimization problems cannot be solved analytically*



## Plan for lectures

Illustrate methods by economic example:
***Portfolio choice model***

**This NOTEBOOK**
- **Model presentation**

- **Numerical Integration** : Evaluate expected value/utility of a given portfolio with stochastic returns
    - Recap: 1-d quadrature
    - Mutidimentional Guassian quadrature
    - Monte Carlo: Simulation based integration (breaks the curse of dimensionality)

**NEXT**
- **Optimization**: Maximize agents expected utility by choosing optimal allocation of wealth in a number of risky assets
    - Newtons method - similar to what we did when solving non-linear eqations
    - Line Searching
    - Non-gradient based solvers (e.g. Nelder-Mead)


### Portfolio choice: Constrained optimization problem
We consider an investor who allocates current wealth, $W$ across $n$ assets indexed by $i=0,\dots, n-1$. 

$$
\max_{(\omega_0, \dots, \omega_{n-1})} E \left\{u\left(\sum_{i=0}^{n-1} \omega_i Z_i\right)\right\} \\
s.t. \sum_{i=0}^{n-1} \omega_i p_i  - W = 0
$$
where asset $i$ has price $p_i$ and a stochastic future value $Z_i$ whose distribution we will specify later. 

- This is a **$n$-dimensional constrained optimization problem** with one equality constraint
- Expectation operator requires evaluating a **$n$-dimensional integral**




### Portfolio choice: unconstrained optimization problem
We can easily impose the budget constraint by substituting $\omega_0 = W - \sum_{i=1}^{n-1}\omega_i p_i$ into the utility function and eliminating $\omega_0$ from the choice set 

The resulting $n-1$-dimensional ***unconstrained optimization problem*** is

$$
\max_{\omega}  U(\omega) =  \max_{\omega} E \left\{u\left(( W - \sum_{i=1}^{n-1}\omega_i)Z_0  + \sum_{i=1}^{n-1} \omega_i Z_i\right)\right\} \label{eq: pp_unconstrained}
$$

where $\omega=(\omega_1, \dots, \omega_{n-1})$  is the vector of portfolio choices.

For simplicity we will assume that all assets have the same price of $1\$$ and set $W=1$ throughout, so that $\omega_i$ refers to the portfolio share in asset $i$.  


## Utility function

We will assume the Constant Relative Risk Aversion (CRRA) utility function

$$
    u(c)=\frac{c^{1-\gamma}-1}{1-\gamma}+1
$$
 
- $\gamma$ is a parameter that measures the degree of relative risk aversion 
- $\gamma<0$ utility function is convex and agent is *risk lover*
- $\gamma=0$ utility function is linear and agent is *risk neutral*
- $\gamma>0$ utility function is concave and agent is *risk averse*
- $\gamma=1$ utility function $u(c)=1+\log(c)$




## CRRA utility function
<center><img src="img/u.png"  width="900" style="">


In [1]:
# Most of what we do today relies on these libraries
import numpy as np
import random
import scipy.stats
import chaospy  ## needs installation
import matplotlib.pyplot as plt

In [2]:
# CRRA utility function
def u(gamma, c): 
    '''CRRA utility function'''
    if gamma!=1:
        return (c**(1-gamma)-1)/(1-gamma)+1
    else:
        return np.log(c)+1

## Computing expected utility
In order to solve the portfolio choice problem we need to be able to evaluate the objective function for a given portfolio allocation, $\omega$

$$
U(\omega)=E \left\{u\left(( W - \sum_{i=1}^{n-1}\omega_i)Z_0  + \sum_{i=1}^{n-1} \omega_i Z_i\right)\right\} \label{eq: objective_unconstrained}
$$

- Expectation operator $E(\cdot)$ requires that we specify the asset return distribution for assets $i=0, \dots, n-1$. 
- Need to specify deistribution of asset returns, $f(Z)$



### Distribution of asset returns
We initially assume that the future value of asset $i$ is assumed to be *independent across assets* and *log-normally distributed* with mean return $R_i$ and variance $\sigma^2_{Z_i}$ so that 

\begin{eqnarray}
Z_i&=&\exp(\varepsilon_i) \quad \varepsilon_i~\sim N(\mu_i, \sigma_i^2) \\
\mu_i&=&\log(R_i^2/\sqrt{R_i^2 + \sigma_{Z_i}^2})\\ \sigma^2_i&=&\log(1+\sigma^2_{Z_i}/R^2_i).
\end{eqnarray}

As we will se below, it is useful to can use the change of variable to obtain $Z_i$ as function of the quantile $q_i$ 

$$Z_i(q_i)=\exp(\mu_1 + \sigma \Phi^{-1}(q_i))$$

where $\Phi^{-1}(q_i)$ is the inverse CDF of the standard normal distribution and $q_i \sim U(0,1)$ is the quantiles uniformly distributed on the interval $[0,1]$. 



### Log normal asset returns
Density of asset returns function for different values of $(R_i,\sigma^2_{Z_i})$. 

<center><img src="img/pdf_Z.png"  width="1500" style="">


### One safe asset and one risky assets, $n=2$
To further simplify matters we will initially assume 
- asset 0: safe asset with $\sigma_{Z_i}=0$ and $Z_0=R_0$
- asset 1 has log normal returns

We then have
$$
U(\omega_1)=E \left\{u\left(R_0(W - \omega_1)  + \omega Z_1)\right)\right\} 
$$

This is the mean of a function of one log-normally distributed variable and only requires to evaluate a one dimensional 

$$
U(\omega_1)=\int_0^{\infty} u\left(R_0(W - \omega_1) + \omega_1 Z_1 \right) g(Z_1)dZ_1  
$$

where $g(Z_1)={\displaystyle {\frac {1}{Z_1\sigma_1 {\sqrt {2\pi }}}}\ \exp \left(-{\frac {\left(\ln Z_1-\mu_1 \right)^{2}}{2\sigma_1 ^{2}}}\right)}$ is the log normal density

### How to solve integral?
We will use **Gauss-Legendre quadrature**

$$
\int_{-1}^1 f(x) dx \approx \sum_{i=1}^{n} w_i f(x_i) 
$$

or more generally
$$
{\displaystyle \int _{a}^{b}f(x)\,dx\approx {\frac {b-a}{2}}\sum _{i=1}^{n}w_{i}
f\left({\frac {(x_i+1)(b-a)}{2} +a }\right)\,dx .}
%f\left({\frac {b-a}{2}}x _{i}+{\frac {a+b}{2}}\right).}
$$



- Nodes and weights $(x_i, w_i)$ come from Legendre polynomials, values tabulated  
- Our integral has domain, $[0, \infty]$, but can be transformed to domain $[0,1]$ using a change of varible based on the inverse cdf



### Change of variable using inverse cdf
Using the change of variable  $Z_1(q_1)=\exp(\mu_1 + \sigma \Phi^{-1}(q_1))$ we obtain a one dimensional integral on the unit interval

\begin{eqnarray}
U(\omega_1)&=&E \left\{u\left(R_0(W - \omega_1)  + \omega_1 \exp(\mu_1 + \sigma_1 \Phi^{-1}(q_1))\right)\right\} \label{eq: Up}  \\
&=&\int_0^1 \left\{u\left(R_0(W - \omega_1)  + \omega_1 \exp(\mu_1 + \sigma_1 \Phi^{-1}(q_1))\right)\right\}dq_1  \nonumber \\
&=&\int_0^1 f(q_1;\omega_1)dq_1  \nonumber
\end{eqnarray}

where

\begin{eqnarray}
f(q_1;\omega_1)=u\left(R_0(W - \omega_1)  + \omega_1 \exp(\mu_1 + \sigma_1 \Phi^{-1}(q_1))\right) 
\end{eqnarray}

is the ex-post utility of receiving an asset return from the $q_1'th$ percentile in the distribution of asset returns.  



### Computing expected utiliy using Gauss-Legendre quadrature
Using Gauss-Legendre quadrature we can then approximate $U(\omega_1)$ as

\begin{eqnarray}
U(\omega_1)& \approx &\sum_{j=1}^m 1/2 w^{[-1,1]}_j f((x^{[-1,1]}_j+1)/2;\omega_1) \nonumber \\
& = & \sum_{j=1}^m  w_j f(x_j;\omega_1) \nonumber 
\end{eqnarray}

where 
- $f(q_1;\omega_1)=u\left(R_0(W - \omega_1)  + \omega_1 \exp(\mu_1 + \sigma_1 \Phi^{-1}(q_1))\right) $
- $w_j=(b-a)/2*w_j^{[-1,1]}$ and $x_j=(x_j^{[-1,1]} + 1)(b-a)/2 +a $ are weights and nodes adjusted to the more general bounded interval $[a,b]$ with a=0 and b=1
- $w_j^{[-1,1]}$  and $x_j^{[-1,1]}$ are Gauss-Legendre weights and nodes for integration on the interval $[-1,1]$. 


In [3]:
def quad(g, order=10, d=1, rule='gauss_legendre', a=0, b=1, output=True, true_val=None):
    '''Compute the integral on [0, 1]^d using qudrature'''

    # generate a d x N matrix of uniform uqdrature nodes where N=(order+1)**d
    distribution = chaospy.Iid(chaospy.Uniform(a, b), d)
    x, w = chaospy.generate_quadrature(order, distribution, rule, sparse=False)
        
    Qn = np.sum(g(x)*w)
    
    if output == True: 
        print('\nResults from quadrature');
        print('Order of polynomial : ', order);
        print('Number of quadrature points : ', w.shape[0]);
        print('Estimate           : ', Qn.round(10));
        
    if true_val!=None: 
        print('True value         : ', true_val)
        print('Approximation error: ', (Qn-true_val).round(10))

    return Qn


### Monte Carlo
Alternatively, we can use simple Monte Carlo integration to approximate $U(\omega)$ 
\begin{eqnarray}
U(\omega)& \approx &1/m\sum_{j=1}^m f(x^j; \omega) \nonumber
\end{eqnarray}
where $x^j=(x^j_0 \dots, x^j_{n-1})$ is $n-$vector of pseudo random draws from the uniform distribution. 

- Note for our one dimensional case with only one risky asset, we only need one random componet, so $x^j$ is a scalar

- Very similar to Gauss-Legendre.. except that the $m$ weights are just w=1/m and nodes are replaced by $m$ draws from the uniform distribution with support $[0,1]^m$

- We will look at the theoretical properties of this later




In [4]:
def monte_carlo(g, N=1000, d=1, rule='random', a=0, b=1, output=True, true_val=None):
    '''Compute the integral I=int_a^b g(x)dx on [a, b]^d using Monte Carlo with sample size N'''

    x0=chaospy.generate_samples(order=N, domain=d, rule=rule)
    w=1/N;
    x=(b-a)*x0+a
       
    gx=g(x)
    
    Qn = np.sum(gx*w)
    se_Qn=np.std(gx)/np.sqrt(N)
    
    if output == True:         
        print('\nResults from Monte Carlo integration');
        print('Number of Monte Carlo draws   : ', N);
        print('Estimate           : ', Qn.round(10));
        print('Standard error     : ', se_Qn.round(10));
    
    if true_val!=None: 
        print('True value         : ', true_val)
        print('Approximation error: ', (Qn-true_val).round(10))

    return Qn, se_Qn

### Numerical Example, $n=2$ 
Risk averse agent divide her wealth, $W$ between $n=2$ assets
- Asset $i=0$ is a safe asset
- Asset $i=1$ is a risky asset with higher return. 


Parameters are: 

- Initial wealth: $W=1$  
- Risk aversion parameter: $\gamma=2$
- Mean returns: $E(Z)=(R_0,R_1)=(1.0,1.4)$
- Variance on returns: $\sigma^2_Z=(\sigma^2_{Z_0},\sigma^2_{Z_1})=(0, 1)$



In [5]:
# Compute expected returns and expected utility for model with one risky asset and one safe asset

# model parameters
n_quad=99      # order of quadrture approximation - results in n_quad+1 nodes
n_mc=100000      # number of monte carlo samlples
W=1            # initial wealth 
gamma=2        # CRRA parameter
omega1=.5      # portfolio allocation in asset 1
R0=1.1;        # Safe return to asset 0
R1=1.4         # Expected return to asset 1 
sigma2_Z1=1**2 # Variance on return to asset 1

# parameters in log-normal
mu1=np.log(R1**2/((R1**2 + sigma2_Z1)**0.5))
sigma1=np.log(1+(sigma2_Z1/R1**2))**0.5
    
# Z1_q: Asset return evaluated at quantile q\in [0,1]
Z1_q =  lambda q: np.exp(mu1+sigma1*scipy.stats.norm.ppf(q))

# u_q utility evaluated at quantile q \in [0,1]
u_q  =  lambda q: u(gamma,  R0*(W-omega1) + omega1*Z1_q(q))

print('\nParameters\n*******************************')
print('nmu1              : ', mu1.round(4));
print('sigma1            : ', sigma1.round(4));
print('Share in asset 1  : ', omega1);

print('\nExpected return on asset 1\n*******************************')
EZ1_quad=quad(Z1_q, n_quad, d=1, a=0, b=1, rule='gauss_legendre', true_val=R1)
EZ1_mc, se_EZ1_mc = monte_carlo(Z1_q, n_mc, d=1, a=0, b=1, rule='random', true_val=R1)

# expected utility
print('\nExpected utility\n*******************************')
EZ1_quad=quad(u_q , n_quad, d=1, a=0, b=1, rule='gauss_legendre')
EZ1_mc, se_EZ1_mc = monte_carlo(u_q , n_mc, d=1, a=0, b=1, rule='random')




Parameters
*******************************
nmu1              :  0.1303
sigma1            :  0.6421
Share in asset 1  :  0.5

Expected return on asset 1
*******************************

Results from quadrature
Order of polynomial :  99
Number of quadrature points :  100
Estimate           :  1.3998485712
True value         :  1.4
Approximation error:  -0.0001514288

Results from Monte Carlo integration
Number of Monte Carlo draws   :  100000
Estimate           :  1.3972130923
Standard error     :  0.0031421802
True value         :  1.4
Approximation error:  -0.0027869077

Expected utility
*******************************

Results from quadrature
Order of polynomial :  99
Number of quadrature points :  100
Estimate           :  1.1054590663

Results from Monte Carlo integration
Number of Monte Carlo draws   :  100000
Estimate           :  1.1051005933
Standard error     :  0.0008451343


### Gauss-Legendre approximation of E(Z). 

-How many quadrature points are needed to give error less than $0.01$? 

<center><img src="img/E_Z.png"  width="1500">


### Gauss-Legendre approximation of $E_m[U(\omega_1=0.5)]$

    
- How does the non-linearity of the expected utility function affect the precision of the approximation? 
- Why is so few nodes needed for $\gamma=1$?

<center><img src="img/E_u_nodes.png"  width="600">
    


### Optimal porfolio, $\omega_1^*=\arg\max_{\omega_1}E[u(\omega_1)]$
- How does the optimal share in the risky asset $\omega_1^*$ vary with risk aversion parameter $\gamma$? 
- We will later use ***optimization methods*** to find $\omega_1$
.... but always good to visualize what you are trying to solve
<center><img src="img/E_u_omega.png"  width="600">
   

# More on Numerical integration
Before we move on to the case with multiple assets, we need to study a numerical integration in a bit more detail

1. **Sumlation and Monte Carlo integration** (see MonteCarlo.ipynb)
1. **Gaussian quadrature** (see MonteCarlo.ipynb)


### The multivariate assets case
For an independent log-normally distributed variable, we also make the change of variable  $Z_i(q_i)=\exp(\mu_i + \sigma \Phi^{-1}(q_i))$ to obtain 
\begin{eqnarray}
U(\omega)&=&E \left\{u\left(( W - \sum_{i=2}^n\omega_i)Z_0(q_0)  + \sum_{i=1}^n \omega_i Z_i(q_i)\right)\right\} \nonumber \\
&=&\int_0^1 \dots \int_0^1 \left\{u\left(( W - \sum_{i=2}^n\omega_i)Z_0(q_0)  + \sum_{i=1}^n \omega_i Z_i(q_i)\right)\right\}dq_0 \dots dq_{n-1} \nonumber \\
&=&\int_{[0,1]^n} f(q;\omega)dq  \nonumber
\end{eqnarray}
where $q=(q_0, \dots, q_{n-1})$ the the vector of uniformly distributed quantiles in the asset return distribution and 
\begin{eqnarray}
f(q;\omega)=u\left(( W - \sum_{i=2}^n\omega_i)Z_0(q_0)  + \sum_{i=1}^n \omega_i Z_i(q_i)\right)
\end{eqnarray}
is the ex-post utility of receiving an asset return from the $q'th$ percentile in the distribution of asset returns.  



### Using Gauss-Legendre quadrature we can then approximate $U(\omega)$ as
\begin{eqnarray}
U(\omega)& \approx &\sum_{j_0=1}^{m_0} \dots \sum_{j_{n-1}=1}^{m_{n-1}}   w^0_{j_0}  \dots w^{n-1}_{j_{n-1}}  f(x_{j_0}^{0},  \dots, x^{n-1}_{j_{n-1}};\omega)
\end{eqnarray}
where 

- $w^i_{j_i}$  and $x^{i}_{j_{i}}$ are Gauss-Legendre weights and nodes for integration dimension $i$ (i.e. asset $i$) adjusted for integration over the interval $[0,1]$ rather than $[-1,1]$. 
- Since Gauss-Legendre is for integration over the interval $[0,1]$ rather than $[-1,1]$, we need to adjust weights $w^{[-1,1]}$ and nodes $x^{[-1,1]}$ for integration on $[a,b]=[0,1]$. 
- The relevant change of variable to the interval $[a,b]$ is $w=(b-a)/2*w^{[-1,1]}$ for the weights and $x=(x^{[-1,1]} + 1)(b-a)/2 +a $ for nodes. 


## Monte Carlo
Alternatively, we can use simple Monte Carlo integration to approximate $U(\omega)$ 
\begin{eqnarray}
U(\omega)& \approx &1/m\sum_{j=1}^m f(x^j; \omega) \nonumber
\end{eqnarray}
where $x^j=(x^j_0 \dots, x^j_{n-1})$ is $n-$vector of pseudo random draws from the uniform distribution. 

- Note for our one dimensional case with only one risky asset, we only need one random componet, so $x^j$ is a scalar

- Very similar to Gauss-Legendre.. except that the $m$ weights are just w=1/m and nodes are replaced by $m$ draws from the uniform distribution with support $[0,1]^m$

- **BREAKS curse of dimensionality**: The error diminishes with a rate $1/\sqrt{m}$ which is idenependent of the dimension of $x$ 



### Example: model with two risky assets 
Parameters:
- Mean returns, $E(Z)=R=(R_0,R_2)=(1.1,1.4)'$
- Variance on returns, $\sigma^2_Z=(\sigma^2_{Z_0},\sigma^2_{Z_1})=(0.5,1)'$
- With and without correlated returns, $Z_i$ between assets

Here we need to do a 2 dimensional Gauss-Legendre quadrature. 
       


In [6]:
# Compute expected returns and expected utility for model with n>2 risky assets and one safe asset

# model parameters
W=1            # initial wealth 
n_quad=9       # order of quadrture approximation - results in n_quad+1 nodes for each asset
n_mc=10000     # number of monte carlo samlples
gamma=2        # CRRA parameter

R=np.array([1.1, 1.4]);  # Expected return to assets i=0,..,n-1
sigma_Z=np.array([0.5, 1]) # standard dev. on return to asset i=0,..,n-1
n_assets = len(R)          # number of assets

n_assets=5;
R=np.ones((n_assets, 1))*1.4
sigma_Z=np.ones((n_assets, 1))*1
sigma_Z[0]=0.5 # standard dev. on return to asset i=0,..,n-1
c=0.9          # mutual corelation between random returns

print('Number of assets:', n_assets)

# portfolio allocation in asset 1,..,n-1
omega=np.ones((n_assets-1, 1))/n_assets 

print('\nParameters in distribution of log asset returns\n*************************************************')
# parameters in (marginal) log-normal
mu=np.log(R**2/((R**2 + sigma_Z**2)**0.5))
mu.shape=(n_assets, 1)
print('Mean, mu:\n ', mu.round(4));

sigma=np.log(1+(sigma_Z/R)**2)**0.5;
sigma.shape=(n_assets, 1)
print('Std. dev (log asset return), sigma:\n ', sigma.round(4));

# build covariance matrix
corr=(1-c)*np.identity(n_assets) + c*np.ones((n_assets,n_assets)); # correlation martrix
print('Correlation martrix (log asset return), corr:\n',corr)

sigma2=sigma*corr*sigma.T                         # covariance martrix
print('Covariance martrix, (log asset return), sigma2:\n',sigma2)

# Lower triangular cholesky matrix, with property: L @ L.T=sigma2 (i.e. "square root" of matrix)
L=np.linalg.cholesky(sigma2)
print('Cholesky matrix, (log asset return), L:\n',L)

# Z1_q: Asset return evaluated at quantile q\in [0,1]
Z_q =  lambda q: np.exp(mu+L @ scipy.stats.norm.ppf(q))

def cv_portfolio(Z_q, q, omega):
    # inputs: 
    # Z_q: n_asset x m
    # omega: n_asset-1 x 1
    # output: 
    # cv: consumption value of porfolio (1 x m)
    
    omega=np.append(W-np.sum(omega), omega)
    omega.shape=(len(omega), 1) # n_assets x 1 
    cv=omega.T@  Z_q(q) 
    return cv

def u_portfolio(Z_q, q, omega, gamma):
    c=cv_portfolio(Z_q, q, omega)
    return u(gamma, c)

q_median=np.ones((n_assets,1))*.5

print('consumption value of portfolio - at median return on all assets\n', cv_portfolio(Z_q, q_median, omega))
print('utility value of portfolio  - at median return on all assets\n', u_portfolio(Z_q, q_median, omega, gamma))

# u_q utility evaluated at quantile q \in [0,1]
cv_q  =  lambda q: u_portfolio(Z_q, q, omega, gamma);
u_q  =  lambda q: u_portfolio(Z_q, q, omega, gamma);

print('Share in asset 1,..n,  : ', omega);

print('\nExpected return on portfolio\n*******************************')
EZ1_quad=quad(cv_q, n_quad, d=n_assets, a=0, b=1, rule='gauss_legendre')
EZ1_mc, se_EZ1_mc = monte_carlo(cv_q, n_mc, d=n_assets, a=0, b=1, rule='random')

# expected utility
print('\nExpected utility\n*******************************')
EZ1_quad=quad(u_q , n_quad, d=n_assets, a=0, b=1, rule='gauss_legendre')
EZ1_mc, se_EZ1_mc = monte_carlo(u_q , n_mc, d=n_assets, a=0, b=1, rule='random')

Number of assets: 5

Parameters in distribution of log asset returns
*************************************************
Mean, mu:
  [[0.2764]
 [0.1303]
 [0.1303]
 [0.1303]
 [0.1303]]
Std. dev (log asset return), sigma:
  [[0.3465]
 [0.6421]
 [0.6421]
 [0.6421]
 [0.6421]]
Correlation martrix (log asset return), corr:
 [[1.  0.9 0.9 0.9 0.9]
 [0.9 1.  0.9 0.9 0.9]
 [0.9 0.9 1.  0.9 0.9]
 [0.9 0.9 0.9 1.  0.9]
 [0.9 0.9 0.9 0.9 1. ]]
Covariance martrix, (log asset return), sigma2:
 [[0.12004804 0.20021547 0.20021547 0.20021547 0.20021547]
 [0.20021547 0.4122448  0.37102032 0.37102032 0.37102032]
 [0.20021547 0.37102032 0.4122448  0.37102032 0.37102032]
 [0.20021547 0.37102032 0.37102032 0.4122448  0.37102032]
 [0.20021547 0.37102032 0.37102032 0.37102032 0.4122448 ]]
Cholesky matrix, (log asset return), L:
 [[0.3464795  0.         0.         0.         0.        ]
 [0.57785663 0.27986874 0.         0.         0.        ]
 [0.57785663 0.1325694  0.24647893 0.         0.        ]
 [0.5778566